In [6]:
import sys
import os
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
from detectron2.utils.logger import setup_logger
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import DatasetCatalog

In [7]:
setup_logger()
 # For COCO format:
register_coco_instances("my_dataset_train", {}, "dataset/instances_minitrain2017.json", "dataset/coco_minitrain2017_25k/images")
# register_coco_instances("my_dataset_val", {}, "dataset/instances_minitrain2017.json", "dataset/coco_minitrain2017_25k/images")
# Get metadata (optional but recommended)
my_dataset_metadata_train = MetadataCatalog.get("my_dataset_train")
# my_dataset_metadata_val = MetadataCatalog.get("my_dataset_val")
cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_1x.yaml")) # Or your modified config file
cfg.merge_from_file("retinanet.yaml")
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
# # List all registered datasets
# print(DatasetCatalog.get("my_dataset_train")[0])
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_1x.yaml") # Load pretrained weights
cfg.MODEL.RETINANET.NUM_CLASSES = 80 # Or the number of classes in your dataset
cfg.OUTPUT_DIR = "./output"

cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.01
cfg.SOLVER.MAX_ITER = 30000
cfg.SOLVER.STEPS = (20000, 25000)

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for inference



Loading config detectron2/model_zoo/configs/Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


In [6]:
#visualize training data
import cv2
from google.colab.patches import cv2_imshow
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

ModuleNotFoundError: No module named 'google.colab'

In [23]:
import cv2
img = cv2.imread('detectron2/dataset/coco_minitrain2017_25k/images/000000012750.jpg')
cv2.imshow(img[:, :, ::-1])

[ WARN:0@1695.744] global loadsave.cpp:268 findDecoder imread_('detectron2/dataset/coco_minitrain2017_25k/images/000000012750.jpg'): can't open/read file: check file path/integrity


TypeError: 'NoneType' object is not subscriptable

In [8]:
import torch
torch.cuda.empty_cache()
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False) # Set resume=True to continue training
trainer.train()

[03/29 03:08:44 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


[03/29 03:08:48 d2.engine.train_loop]: Starting training from iteration 0
ERROR [03/29 03:08:48 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "/home/pankaj/Downloads/cs776/proj/detectron2/detectron2/engine/train_loop.py", line 155, in train
    self.run_step()
  File "/home/pankaj/Downloads/cs776/proj/detectron2/detectron2/engine/defaults.py", line 530, in run_step
    self._trainer.run_step()
  File "/home/pankaj/Downloads/cs776/proj/detectron2/detectron2/engine/train_loop.py", line 310, in run_step
    loss_dict = self.model(data)
                ^^^^^^^^^^^^^^^^
  File "/home/pankaj/Downloads/cs776/proj/proj776/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pankaj/Downloads/cs776/proj/proj776/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_c

OutOfMemoryError: CUDA out of memory. Tried to allocate 54.00 MiB. GPU 0 has a total capacity of 5.68 GiB of which 32.88 MiB is free. Including non-PyTorch memory, this process has 5.62 GiB memory in use. Of the allocated memory 5.43 GiB is allocated by PyTorch, and 74.78 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)